# Canny Edge kernel for Video Pipeline

Load overlay where input to ip = 8b and output is 32b padded grayscaled pixels. RGBA = GRAY * 0b00010101. Attempts to alter the VDMA to 8 bit output have failed. Relation between pixel depth and resolution is unclear. A bug is whenever some other feature is loaded the screen tends to 'roll'; there is no fixation point. Todo: fix bug, implement changing different kernels on the fly, improve sequential algorithm for comparison, other functions of canny edging. 

In [15]:
from pynq import Overlay
from pynq.lib.video import *
from pynq.lib import *

base = Overlay('kernelmult_p32b_for_1280x720.bit', ignore_version=True)
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out


Configure HDMI

In [21]:
Mode = VideoMode(1280,720,24)
hdmi_in.configure(PIXEL_RGB)
#hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
print(hdmi_in.colorspace)


# monitor configuration: 1280*720 @ 60Hz ;32 voor raspbpi configged at 720p 50hz
Mode2 = VideoMode(1280,720,8)
print(hdmi_in.mode)
print(hdmi_out.mode)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode2, PIXEL_GRAY)
hdmi_out.start()

hdmi_in.tie(hdmi_out)
print(hdmi_out.colorspace)



[-0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0]
VideoMode: width=1280 height=718 bpp=24
VideoMode: width=1280 height=720 bpp=8
[1.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [ ]:
Gepruts met colorspace conversion. Niet uitvoeren voor normaal canny edging

In [ ]:
colorspace_in = base.video.hdmi_in.color_convert
colorspace_out = base.video.hdmi_out.color_convert

bgr2rgb = [-1, -1, -1,
           -1, 8, -1, 
           -1, -1, -1,
           0, 0, 0]

brg2gray = [0.11328125, 0.5859375, 0.296875, 0.5, -0.328125, -0.16796875, -0.078125, -0.41796875, 0.5, -0.0, 0.5, 0.5]

colorspace_in.colorspace = brg2gray
colorspace_out.colorspace = bgr2rgb

colorspace_in.colorspace

print(hdmi_out.colorspace)


Gepruts met pixelpack conversion. Niet uitvoeren voor normaal canny edging

In [ ]:
pixel_in = base.video.hdmi_in.pixel_pack
pixel_out = base.video.hdmi_out.pixel_unpack

pixel_in.bits_per_pixel = 8
pixel_out.bits_per_pixel = 8

pixel_in.bits_per_pixel


Write configuration data (for the `hfilt` function)

In [11]:
from pynq.mmio import MMIO
msk = MMIO(0x43C7_0000, 0x1000)
#c0:edge
#c1:impulse
#c2:blur
#c3:sobel
#c4:tie
msk.write(0x10, 0x0)
#normalfactor for pixel division
msk.write(0x18, 0x4)
#pixelchannelselector
#0:01010101, 1:00010101, 2:00000101, 3:00000001, 4:empty
msk.write(0x20, 0x0)

Don't forget to close the HDMI connection!

In [ ]:
hdmi_in.tie(hdmi_out)

In [40]:
hdmi_out.close()
hdmi_in.close()
